In [2]:
import pandas as pd
df = pd.read_csv('hbp3.tsv', delimiter='\t')

In [3]:
df.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),DATENAME,DATETIME,HOURS,Table Name,building,floor,sensorId,stall,status,unixtimestamp
0,2017/12/20 13:13,Wednesday,13,Wednesday,2017/12/20 4:13,4,1513743187638.json,91,4,910401,1,close,1.510000e+12
1,2017/12/20 13:16,Wednesday,13,Wednesday,2017/12/20 4:16,4,1513743383454.json,91,4,910403,3,close,1.510000e+12
2,2017/12/20 13:21,Wednesday,13,Wednesday,2017/12/20 4:21,4,1513743674523.json,91,4,910402,2,close,1.510000e+12
3,2017/12/20 13:23,Wednesday,13,Wednesday,2017/12/20 4:23,4,1513743794829.json,91,4,910403,3,open,1.510000e+12
4,2017/12/20 13:23,Wednesday,13,Wednesday,2017/12/20 4:23,4,1513743835781.json,91,4,910403,3,close,1.510000e+12


In [4]:
unixt = df["Table Name"].str.extract('([0-9]+)')

C:\Users\shoichi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [5]:
dat = df.loc[:,["DATETIME(JST)","DATENAME(JST)","HOURS(JST)","sensorId","stall","status"]]

In [6]:
dat = pd.concat([dat, unixt], axis=1)
#カラムネームの変更を行う
columns={
"Table Name":"unixtimestamp"
}
#再度読み込みを行う
dat.rename(columns=columns, inplace=True)
dat.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),sensorId,stall,status,unixtimestamp
0,2017/12/20 13:13,Wednesday,13,910401,1,close,1513743187638
1,2017/12/20 13:16,Wednesday,13,910403,3,close,1513743383454
2,2017/12/20 13:21,Wednesday,13,910402,2,close,1513743674523
3,2017/12/20 13:23,Wednesday,13,910403,3,open,1513743794829
4,2017/12/20 13:23,Wednesday,13,910403,3,close,1513743835781


In [11]:
dat1 = dat[dat.stall==1]
dat2 = dat[dat.stall==2]
dat3 = dat[dat.stall==3]

In [15]:
for i in range(len(dat1)):
    try:
        if dat1.iloc[i]["status"] == "open" and dat1.iloc[i+1]["status"] == "close":
            print(str(i))
            opn = dat1.iloc[[i]]
            cls = dat1.iloc[[i+1]]
            d = pd.concat([cls, opn])
            d.to_csv("preset/"+str(i)+".csv", index=False)
    except:
        print('indexer is out-of-bounds')

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
253
255
257
259
261
263
265
267
269
271
273
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
317
319
321
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351
353
355
357
359
361
363
365
367
369
371
373
375
377
379
381
383
385
387
389
391
393
395
397
399
401
403
405
407
409
411
413
415
417
419
421
423
425
427
429
431
433
435
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
527
529
531
533

In [23]:
import glob
path = "preset/" # use your path
all_files = glob.glob(path + "*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)
stall1 = frame.sort_values(by=['unixtimestamp'])
stall1.to_csv('stall1.csv', index=None)

In [24]:
cls1 = stall1[stall1.status == "close"].reset_index(drop=True)
op1 = stall1[stall1.status == "open"].reset_index(drop=True)

In [25]:
cls1.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),sensorId,stall,status,unixtimestamp
0,2017/12/20 13:45,Wednesday,13,910401,1,close,1513745121391
1,2017/12/20 13:53,Wednesday,13,910401,1,close,1513745607821
2,2017/12/20 14:03,Wednesday,14,910401,1,close,1513746208398
3,2017/12/20 14:08,Wednesday,14,910401,1,close,1513746485029
4,2017/12/20 14:11,Wednesday,14,910401,1,close,1513746669042


In [26]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"close_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst",
    "unixtimestamp":"close_unixtime"
}
#再度読み込みを行う
cls1.rename(columns=columns, inplace=True)

In [27]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"open_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst",
    "unixtimestamp":"open_unixtime"
}
#再度読み込みを行う
op1.rename(columns=columns, inplace=True)

In [28]:
op1["open_time"].head()

0    2017/12/20 13:32
1    2017/12/20 13:50
2    2017/12/20 13:59
3    2017/12/20 14:07
4    2017/12/20 14:10
Name: open_time, dtype: object

In [29]:
cls1["close_time"].head()

0    2017/12/20 13:45
1    2017/12/20 13:53
2    2017/12/20 14:03
3    2017/12/20 14:08
4    2017/12/20 14:11
Name: close_time, dtype: object

In [30]:
len(op1)

1198

In [38]:
len(cls1)

1198

In [39]:
f1 = pd.concat([op1["open_time"], cls1, op1["open_unixtime"]], axis=1).drop("status", axis=1)

In [40]:
f1.head()

,open_time,close_time,DATENAME,hour_jst,sensorId,stall,close_unixtime,open_unixtime
0,2017/12/20 13:32,2017/12/20 13:45,Wednesday,13,910401,1,1513745121391,1513744358353
1,2017/12/20 13:50,2017/12/20 13:53,Wednesday,13,910401,1,1513745607821,1513745411116
2,2017/12/20 13:59,2017/12/20 14:03,Wednesday,14,910401,1,1513746208398,1513745973139
3,2017/12/20 14:07,2017/12/20 14:08,Wednesday,14,910401,1,1513746485029,1513746448689
4,2017/12/20 14:10,2017/12/20 14:11,Wednesday,14,910401,1,1513746669042,1513746614840


In [41]:
f1.to_csv('f1.csv',index=False)

In [107]:
for i in range(len(dat2)):
    try:
        if dat2.iloc[i]["status"] == "open" and dat2.iloc[i+1]["status"] == "close":
#        print(str(i))
            cls = dat2.iloc[[i]]
            opn = dat2.iloc[[i+1]]
            d = pd.concat([cls, opn])
            d.to_csv("preset2/"+str(i)+".csv", index=False)
    except:
        print('indexer is out-of-bounds')

indexer is out-of-bounds


In [108]:
import glob
path = "preset2/" # use your path
all_files = glob.glob(path + "*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)
stall2 = frame.sort_values(by=['unixtimestamp'])
stall2.to_csv('stall2.csv', index=None)

In [109]:
cls2 = stall2[stall2.status == "close"].reset_index(drop=True)
op2 = stall2[stall2.status == "open"].reset_index(drop=True)

In [110]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"close_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
cls2.rename(columns=columns, inplace=True)

#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"open_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
op2.rename(columns=columns, inplace=True)

In [111]:
f2 = pd.concat([op2["open_time"], cls2], axis=1)
f2.to_csv('f2.csv',index=False)

In [17]:
###openのみをカウントする
df1 = df[df.stall==1]
df2 = df[df.stall==2]
df3 = df[df.stall==3]

In [20]:
df1[df1.status=="open"].reset_index(drop=True).head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),DATENAME,DATETIME,HOURS,Table Name,building,floor,sensorId,stall,status,unixtimestamp
0,2017/12/20 13:32,Wednesday,13,Wednesday,2017/12/20 4:32,4,1513744358353.json,91,4,910401,1,open,1.510000e+12
1,2017/12/20 13:50,Wednesday,13,Wednesday,2017/12/20 4:50,4,1513745411116.json,91,4,910401,1,open,1.510000e+12
2,2017/12/20 13:59,Wednesday,13,Wednesday,2017/12/20 4:59,4,1513745973139.json,91,4,910401,1,open,1.510000e+12
3,2017/12/20 14:07,Wednesday,14,Wednesday,2017/12/20 5:07,5,1513746448689.json,91,4,910401,1,open,1.510000e+12
4,2017/12/20 14:10,Wednesday,14,Wednesday,2017/12/20 5:10,5,1513746614840.json,91,4,910401,1,open,1.510000e+12


In [21]:
d1open = df1[df1.status=="open"].reset_index(drop=True)

In [22]:
len(d1open)

1221

In [24]:
d1out= d1open[d1open.index % 2 == 0].reset_index(drop=True)

In [27]:
d1in= d1open[d1open.index % 2 != 0].reset_index(drop=True)

In [28]:
d1out.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),DATENAME,DATETIME,HOURS,Table Name,building,floor,sensorId,stall,status,unixtimestamp
0,2017/12/20 13:32,Wednesday,13,Wednesday,2017/12/20 4:32,4,1513744358353.json,91,4,910401,1,open,1.510000e+12
1,2017/12/20 13:59,Wednesday,13,Wednesday,2017/12/20 4:59,4,1513745973139.json,91,4,910401,1,open,1.510000e+12
2,2017/12/20 14:10,Wednesday,14,Wednesday,2017/12/20 5:10,5,1513746614840.json,91,4,910401,1,open,1.510000e+12
3,2017/12/20 14:33,Wednesday,14,Wednesday,2017/12/20 5:33,5,1513747983957.json,91,4,910401,1,open,1.510000e+12
4,2017/12/20 14:50,Wednesday,14,Wednesday,2017/12/20 5:50,5,1513749029664.json,91,4,910401,1,open,1.510000e+12


In [29]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"in_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst",
    "unixtimestamp":"in_unixtime"
}
#再度読み込みを行う
d1in.rename(columns=columns, inplace=True)

#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"out_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst",
    "unixtimestamp":"out_unixtime"
}
#再度読み込みを行う
d1out.rename(columns=columns, inplace=True)

In [31]:
oponly = pd.concat([d1in["in_time"], d1out], axis=1)

In [32]:
oponly.to_csv('oponly.csv',index=False)